In [4]:
# ==============================================================================
# --- 0. Preamble: Load Packages ---
# ==============================================================================
import pandas as pd
import numpy as np
import os

# Optional: Notebook display settings for wider tables
from IPython.display import display, HTML
display(HTML("<style>:root { --jp-notebook-max-width: 90% !important; }</style>"))

In [28]:
# *** USER: SELECT WHICH SPECIES TO RUN ***
SPECIES_TO_RUN = "Rice" # Options: "Maize", "Soybean", "Rice"

# --- Define directory paths relative to the notebook's location in `scripts/` ---
data_directory = "../data/"
penalty_directory = "../penalty_functions/"
output_directory = "../results/"

# --- Automatically generate file paths based on the selected species ---
coacr_file = os.path.join(data_directory, f"{SPECIES_TO_RUN.lower()}_coaccessibilty_scores.csv")
penalty_file = os.path.join(penalty_directory, f"{SPECIES_TO_RUN.lower()}_penalty_parameters.tsv")
output_file = os.path.join(output_directory, f"{SPECIES_TO_RUN.lower()}_penalized_coaccess.csv")

# Create the output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

print(f"Selected Species: {SPECIES_TO_RUN}")
print(f"Input Co-accessibility File: {coacr_file}")
print(f"Input Penalty File: {penalty_file}")
print(f"Output File: {output_file}")


Selected Species: Rice
Input Co-accessibility File: ../data/rice_coaccessibilty_scores.csv
Input Penalty File: ../penalty_functions/rice_penalty_parameters.tsv
Output File: ../results/rice_penalized_coaccess.csv


In [29]:
# ==============================================================================
# --- 2. Define Penalty Functions ---
# ==============================================================================

def load_penalty_parameters(filepath):
    """
    Loads the penalty function parameters from a TSV file.
    The file must contain columns for weight, beta, and alpha.
    """
    try:
        params_df = pd.read_csv(filepath, sep='\t')
        
        # Define a mapping from the file's column names to the expected names
        column_mapping = {
            'pi_i (weight)': 'weight',
            'alpha_i (exponent)': 'alpha',
            'beta_i (scaling_factor)': 'beta'
        }
        
        # Rename the columns to match what the script expects
        params_df.rename(columns=column_mapping, inplace=True)

        for col in ['weight', 'beta', 'alpha']:
            if col in params_df.columns:
                # Convert column to string, remove any commas, then convert to numeric
                params_df[col] = pd.to_numeric(params_df[col].astype(str).str.replace(',', ''), errors='coerce')
            else:
                raise ValueError(f"Required column '{col}' not found after renaming.")
        
        # Drop rows where conversion might have failed
        params_df.dropna(subset=['weight', 'beta', 'alpha'], inplace=True)

        if not all(col in params_df.columns for col in ['weight', 'beta', 'alpha']):
            raise ValueError("Penalty file must contain columns for weight, beta, and alpha.")
        return params_df.to_dict('records')
    except FileNotFoundError:
        print(f"Error: Penalty parameter file not found at {filepath}")
        return None
    except Exception as e:
        print(f"Error loading penalty parameters: {e}")
        return None

def calculate_penalty(distance, params):
    """
    Calculates the penalty for a given distance based on the GMM parameters.
    The penalty is the sum of n weighted power-law components.
    P(s) = sum(pi_i * beta_i * s^-alpha_i)
    """
    penalty = 0
    # Ensure distance is a positive number to avoid division by zero or complex numbers
    if distance <= 0:
        return 1.0 # No penalty for zero or negative distance
        
    for p in params:
        penalty += p['weight'] * p['beta'] * (distance ** -p['alpha'])
    return penalty

In [30]:
# ==============================================================================
# --- 3. Load Data ---
# ==============================================================================

print("Loading penalty parameters...")
penalty_params = load_penalty_parameters(penalty_file)

print("Loading co-accessibility data...")
coaccess_df = pd.read_csv(coacr_file)

# Filter for rows with positive co-accessibility scores
coaccess_df = coaccess_df[coaccess_df['coaccess'] > 0].copy()
print(f"Number of pairs with positive co-accessibility: {len(coaccess_df):,}")


print("Data loaded successfully.")
display(coaccess_df.head())

Loading penalty parameters...
Loading co-accessibility data...
Number of pairs with positive co-accessibility: 3,836,372
Data loaded successfully.


,coACR,coaccess,distance
0,Chr1_852_1353-Chr1_10687_11188,0.087854,9835
1,Chr1_852_1353-Chr1_16102_16603,0.163708,15250
2,Chr1_852_1353-Chr1_22848_23349,0.073163,21996
3,Chr1_852_1353-Chr1_26866_27367,0.107967,26014
4,Chr1_852_1353-Chr1_29606_30107,0.063463,28754


In [31]:
# ==============================================================================
# --- 4. Apply Penalty ---
# ==============================================================================

print("Applying penalty function... (This may take a moment for large files)")

# Use the .apply() method on the 'distance' column to calculate penalties
penalties = coaccess_df['distance'].apply(lambda d: calculate_penalty(d, penalty_params))

# Multiply the original co-accessibility score by the calculated penalty
coaccess_df['penalized_coaccess'] = coaccess_df['coaccess'] * penalties

print("Penalty applied successfully.")
display(coaccess_df.head())

Applying penalty function... (This may take a moment for large files)
Penalty applied successfully.


,coACR,coaccess,distance,penalized_coaccess
0,Chr1_852_1353-Chr1_10687_11188,0.087854,9835,1.753690e+06
1,Chr1_852_1353-Chr1_16102_16603,0.163708,15250,7.578407e+05
2,Chr1_852_1353-Chr1_22848_23349,0.073163,21996,1.010217e+05
3,Chr1_852_1353-Chr1_26866_27367,0.107967,26014,8.574967e+04
4,Chr1_852_1353-Chr1_29606_30107,0.063463,28754,3.623808e+04


In [32]:
# ==============================================================================
# --- 5. Save Corrected Data ---
# ==============================================================================

print(f"Saving penalized results to: {output_file}")
coaccess_df.to_csv(output_file, index=False)
print("Done!")

Saving penalized results to: ../results/rice_penalized_coaccess.csv
Done!
